<a href="https://colab.research.google.com/github/haneulllll/Deeplearning/blob/practice/lab3_Regularization_and_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regularization

**Dropout 기법**

**import package**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data_utils
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
import random

**GPU/CPU setting**


1. 상단메뉴의 런타임 - 런타임 유형 변경 - 하드웨어 가속기 메뉴에서 GPU를 선택 - 저장

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

random.seed(777)
torch.manual_seed(777)
if device == 'cuda' :
  torch.cuda.manual_seed_all(777)

cpu


**데이터 불러오기**

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 9912422/9912422 [00:00<00:00, 35595368.91it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 28881/28881 [00:00<00:00, 959384.26it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 1648877/1648877 [00:00<00:00, 10157850.72it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 4542/4542 [00:00<00:00, 7756729.95it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



**데이터를 배치화하기**

minibatch의 크기는 32로 설정

In [ ]:
batch_size = 32
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=False)

**Model 생성**

-- Hidden layer 수 : 3개 (784 - 512 - 256 - 128 - 10)

-- 활성화 함수 : ReLU

-- 정규화 기법 : dropout , Batch normalization 이용

-- Optimization 기법 : Adam

In [ ]:
class MLP_Dropout(nn.Module) :
  def __init__(self) :
    super(MLP_Dropout, self).__init__()
    self.fc1 = nn.Linear(784,512)
    self.fc2 = nn.Linear(512,256)
    self.fc3 = nn.Linear(256,128)
    self.fc4 = torch.nn.Linear(128,10)
    self.dp1 = nn.Dropout(p=0.3)
    self.dp2 = nn.Dropout(p=0.2)
    self.dp3 = nn.Dropout(p=0.1)
    self.bn1 = nn.BatchNorm1d(512)
    self.bn2 = nn.BatchNorm1d(256)
    self.bn3 = nn.BatchNorm1d(128)

  def forward(self, x) :
    h1 = F.relu(self.bn1(self.fc1(x)))
    h1dp = self.dp1(h1)
    h2 = F.relu(self.bn2(self.fc2(h1dp)))
    h2dp = self.dp2(h2)
    h3 = F.relu(self.bn3(self.fc3(h2dp)))
    h3dp = self.dp3(h3)
    output = self.fc4(h3dp)
    return output

In [ ]:
# 파라미터 정의 -> 반복횟수 30으로 설정 / learning_rate는 0.001로 설정

num_epochs = 30
learning_rate = 0.001

In [ ]:
# 모델 정의

model = MLP_Dropout().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

**Train 과정**

In [ ]:
loss_list = []
for epoch_num in range(num_epochs) :
  average_cost = 0
  model.train()

  for batch_idx, (x_data, y_label) in enumerate(train_loader) :
    num_of_mini_batch = len(train_loader)

    x_data = x_data.reshape(-1, 28*28)
    input_image = x_data.to(device)
    label = y_label.to(device)
    optimizer.zero_grad()
    y_predict = model(input_image)
    loss = criterion(y_predict, label)
    loss.backward()
    optimizer.step()

    average_cost = average_cost + (loss.item()/num_of_mini_batch)
    loss_list.append(loss)

  print("Epoch {} Loss {:.5f}".format((epoch_num+1), average_cost))

Epoch 1 Loss 0.27662
Epoch 2 Loss 0.15501
Epoch 3 Loss 0.12642
Epoch 4 Loss 0.10693
Epoch 5 Loss 0.09167
Epoch 6 Loss 0.08518
Epoch 7 Loss 0.07368
Epoch 8 Loss 0.06882
Epoch 9 Loss 0.06585
Epoch 10 Loss 0.05944
Epoch 11 Loss 0.05515
Epoch 12 Loss 0.05375
Epoch 13 Loss 0.04891
Epoch 14 Loss 0.04985
Epoch 15 Loss 0.04078
Epoch 16 Loss 0.04378
Epoch 17 Loss 0.03995
Epoch 18 Loss 0.04135
Epoch 19 Loss 0.03569
Epoch 20 Loss 0.03713
Epoch 21 Loss 0.03619
Epoch 22 Loss 0.03414
Epoch 23 Loss 0.03359
Epoch 24 Loss 0.03188
Epoch 25 Loss 0.03157
Epoch 26 Loss 0.02898
Epoch 27 Loss 0.02871
Epoch 28 Loss 0.02717
Epoch 29 Loss 0.02778
Epoch 30 Loss 0.02695


**Validation 과정**

-- dropout을 적용하지 않으므로 model을 평가모드로 전환

In [ ]:
with torch.no_grad() :
  num_total_data = 0
  correct = 0
  model.eval()
  for batch_idx, (images, labels) in enumerate(test_loader) :

    images = images.reshape(-1, 28*28)
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    outputs_softmax = F.softmax(outputs, dim=1)
    predicted = torch.argmax(outputs_softmax, dim=1)

    num_total_data += len(images)
    answer = sum(labels == predicted).item()
    correct += answer

print("Model accuracy {:.5f}%".format((correct/num_total_data)*100))

Model accuracy 98.60000%
